# PIP


In [7]:
pip install transformers torch scikit-learn pandas tqdm


# GPT


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("augmented_data1_final.csv")  # Ensure CSV has 'text' and 'intent' columns

# Encode labels
df['intent'] = df['intent'].astype('category').cat.codes  # Convert intent to numerical labels

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['instruction_augmented'].tolist(), df['intent'].tolist(), test_size=0.2, random_state=42
)


In [ ]:
!pip install easynmt


  error: subprocess-exited-with-error
  
  × Building wheel for fasttext (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [31 lines of output]
      C:\Users\reaga\AppData\Local\Temp\pip-build-env-nvoz3vvu\overlay\Lib\site-packages\setuptools\dist.py:493: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2025-Mar-03, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        


  Using cached EasyNMT-2.0.2-py3-none-any.whl
  Using cached fasttext-0.9.3.tar.gz (73 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
Failed to build fasttext


# Data augmentaiton


In [ ]:
from easynmt import EasyNMT
print("EasyNMT imported successfully!")


In [ ]:
from googletrans import Translator


# Start


In [12]:


import pandas as pd
from transformers import GPT2Tokenizer
from datasets import Dataset
import ast
import torch

from sklearn.preprocessing import LabelEncoder


In [13]:
import pandas as pd

# Load the dataset from CSV
def load_dataset(csv_file):
    df = pd.read_csv(csv_file)
    if 'instruction' not in df.columns or 'intent' not in df.columns:
        raise ValueError("Dataset must contain 'instruction' and 'intent' columns")
    return df

csv_file = "dataset.csv"
print("Loading dataset...")
df = load_dataset(csv_file)
print("Dataset loaded successfully!")

# --- Analysis before dropping ---
print("\n--- Before Dropping ---")
print(f"Total rows: {len(df)}")
print(f"Missing values in 'instruction': {df['instruction'].isnull().sum()}")

# --- Drop missing values ---
df.dropna(subset=['instruction'], inplace=True)

# --- Analysis after dropping ---
print("\n--- After Dropping ---")
print(f"Total rows: {len(df)}")
print(f"Missing values in 'instruction': {df['instruction'].isnull().sum()}")

# --- Validation ---
if df['instruction'].isnull().sum() == 0:
    print("\nValidation: No missing values in 'instruction' after dropping.")
else:
    print("\nValidation: There are still missing values in 'instruction'.")

# Display the first few rows to verify
print(df.head())


Loading dataset...
Dataset loaded successfully!

--- Before Dropping ---
Total rows: 26872
Missing values in 'instruction': 0

--- After Dropping ---
Total rows: 26872
Missing values in 'instruction': 0

Validation: No missing values in 'instruction' after dropping.
   flags                                        instruction category  \
0      B   question about cancelling order {{Order Number}}    ORDER   
1    BQZ  i have a question about cancelling oorder {{Or...    ORDER   
2   BLQZ    i need help cancelling puchase {{Order Number}}    ORDER   
3     BL         I need to cancel purchase {{Order Number}}    ORDER   
4  BCELN  I cannot afford this order, cancel purchase {{...    ORDER   

         intent                                           response  
0  cancel_order  I've understood you have a question regarding ...  
1  cancel_order  I've been informed that you have a question ab...  
2  cancel_order  I can sense that you're seeking assistance wit...  
3  cancel_order  I under

In [14]:
import torch
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification

# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Load GPT-2 model for sequence classification (instead of just the base model)
gpt2_model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=len(df['intent'].unique()))

# Move the model to the GPU if available, otherwise, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt2_model.to(device)

# Get the number of unique intents
num_intents = len(df['intent'].unique())
print(f"Number of unique intents: {num_intents}")



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of unique intents: 27


In [15]:
# Encode labels for intent classification only
def encode_labels(df):
    # Create a mapping for intent labels
    intent_map = {intent: idx for idx, intent in enumerate(df['intent'].unique())}

    # Reverse mapping for decoding (optional)
    reverse_intent_map = {v: k for k, v in intent_map.items()}

    # Encode 'intent' column
    df['intent_encoded'] = df['intent'].map(intent_map)

    return df, intent_map, reverse_intent_map

# Encode labels
print("Encoding labels...")
df, intent_map, reverse_intent_map = encode_labels(df)

# Print intent mappings
print("\nIntent Mapping:")
print(intent_map)


Encoding labels...

Intent Mapping:
{'cancel_order': 0, 'change_order': 1, 'change_shipping_address': 2, 'check_cancellation_fee': 3, 'check_invoice': 4, 'check_payment_methods': 5, 'check_refund_policy': 6, 'complaint': 7, 'contact_customer_service': 8, 'contact_human_agent': 9, 'create_account': 10, 'delete_account': 11, 'delivery_options': 12, 'delivery_period': 13, 'edit_account': 14, 'get_invoice': 15, 'get_refund': 16, 'newsletter_subscription': 17, 'payment_issue': 18, 'place_order': 19, 'recover_password': 20, 'registration_problems': 21, 'review': 22, 'set_up_shipping_address': 23, 'switch_account': 24, 'track_order': 25, 'track_refund': 26}


In [16]:
# Save the preprocessed dataset as JSON
def save_preprocessed_dataset(df, json_file):
    # Convert DataFrame to JSON format
    df.to_json(json_file, orient='records', lines=True)

# Path to save the preprocessed JSON file
json_file = "preprocessed_dataset.json"  # Replace with your desired output path

# Save the preprocessed dataset
print("\nSaving preprocessed dataset as JSON...")
save_preprocessed_dataset(df, json_file)
print(f"Preprocessed dataset saved to {json_file}")



Saving preprocessed dataset as JSON...
Preprocessed dataset saved to preprocessed_dataset.json


In [18]:
# Load the preprocessed dataset from JSON for intent classification only
print("Loading preprocessed dataset from JSON...")
preprocessed_df = pd.read_json(json_file, lines=True)

# Display the first few rows of the preprocessed dataset
print("\nPreprocessed Dataset:")
print(preprocessed_df.head())

# Verify the encoded 'intent' column
print("\nEncoded Intent Example:")
print(preprocessed_df[['instruction', 'intent', 'intent_encoded']].head())


Loading preprocessed dataset from JSON...

Preprocessed Dataset:
   flags                                        instruction category  \
0      B   question about cancelling order {{Order Number}}    ORDER   
1    BQZ  i have a question about cancelling oorder {{Or...    ORDER   
2   BLQZ    i need help cancelling puchase {{Order Number}}    ORDER   
3     BL         I need to cancel purchase {{Order Number}}    ORDER   
4  BCELN  I cannot afford this order, cancel purchase {{...    ORDER   

         intent                                           response  \
0  cancel_order  I've understood you have a question regarding ...   
1  cancel_order  I've been informed that you have a question ab...   
2  cancel_order  I can sense that you're seeking assistance wit...   
3  cancel_order  I understood that you need assistance with can...   
4  cancel_order  I'm sensitive to the fact that you're facing f...   

   intent_encoded  
0               0  
1               0  
2               0  
3

In [19]:
from transformers import GPT2TokenizerFast
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch

# Load GPT-2 tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token

# Define a custom dataset class for intent classification
class IntentClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['instruction']
        intent_label = row['intent_encoded']

        # Tokenize the input
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'intent_labels': intent_label
        }

# Create datasets (train/test split)
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Create dataset objects
train_dataset = IntentClassificationDataset(train_data, tokenizer, max_length=128)
val_dataset = IntentClassificationDataset(val_data, tokenizer, max_length=128)

# Create DataLoader objects
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [20]:
import torch
import torch.nn as nn
from transformers import GPT2Model

class IntentGPT2(nn.Module):
    def __init__(self, gpt2_model, num_intents, dropout_prob=0.4):
        super(IntentGPT2, self).__init__()
        self.gpt2 = gpt2_model

        self.dropout = nn.Dropout(dropout_prob)  # Dropout for intent only

        self.intent_head = nn.Linear(768, num_intents)  # Classification head for intent

    def forward(self, input_ids, attention_mask):
        # Forward pass through GPT-2 model
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

        # Mean pooling for intent classification
        cls_like_embedding = torch.mean(last_hidden_state, dim=1)
        cls_like_embedding = self.dropout(cls_like_embedding)  # Apply dropout

        # Intent classification
        intent_logits = self.intent_head(cls_like_embedding)

        return intent_logits


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, reduction='mean'):
        """
        Focal Loss for intent classification to handle class imbalance.

        Args:
            alpha (Tensor, optional): Class weights (inverse frequency) for handling imbalance.
            gamma (int, optional): Focusing parameter to adjust difficulty weighting (default: 2).
            reduction (str, optional): Specifies 'mean', 'sum', or 'none' reduction (default: 'mean').
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class weights (optional, computed from inverse frequency)
        self.gamma = gamma  # Focusing parameter to handle difficult examples
        self.reduction = reduction  # Specifies how the loss should be reduced ('mean', 'sum', 'none')

    def forward(self, inputs, targets):
        """
        Compute the focal loss for intent classification.

        Args:
            inputs (Tensor): Model predictions (logits), shape: (batch_size, num_classes).
            targets (Tensor): Ground truth labels, shape: (batch_size).

        Returns:
            Tensor: Computed focal loss.
        """
        # Convert logits to probabilities using softmax
        log_probs = F.log_softmax(inputs, dim=-1)  # Log probabilities
        probs = torch.exp(log_probs)  # Convert log_probs to probabilities

        # Gather log_probs corresponding to the target class
        targets = targets.view(-1, 1)  # Ensure correct shape for targets
        log_pt = log_probs.gather(dim=-1, index=targets).squeeze(-1)  # Log-prob for target class
        pt = probs.gather(dim=-1, index=targets).squeeze(-1)  # Probability for target class

        # Compute the focal loss modulating factor
        focal_weight = (1 - pt) ** self.gamma  # The modulating factor (1 - pt) ^ gamma

        # Compute focal loss
        focal_loss = -focal_weight * log_pt  # Apply modulating factor to log_pt

        # Apply class weights (alpha) if provided
        if self.alpha is not None:
            alpha_factor = self.alpha[targets.squeeze(-1)]  # Weights for target class
            focal_loss = alpha_factor * focal_loss  # Apply class weighting

        # Apply reduction (mean, sum, or none)
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss  # No reduction, return per sample loss


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

# ============================
# HYPERPARAMETERS & SETUP
# ============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("DEvice", device)
learning_rate = 5e-5
num_epochs = 5



# Number of folds for cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# ============================
# COMPUTE CLASS WEIGHTS
# ============================
num_intents = len(df['intent_encoded'].unique())

# Compute intent class weights
intent_weights = compute_class_weight('balanced', classes=np.arange(num_intents), y=df['intent_encoded'])

# Convert to tensor and move to GPU
intent_weights = torch.tensor(intent_weights, dtype=torch.float).to(device)

# ============================
# LOSS FUNCTIONS
# ============================
intent_loss_fn = nn.CrossEntropyLoss(weight=intent_weights)

# ============================
# MODEL SETUP
# ============================
gpt2_model = GPT2Model.from_pretrained('gpt2')
model = IntentGPT2(gpt2_model, num_intents).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# ============================
# TRAINING FUNCTION
# ============================
scaler = torch.cuda.amp.GradScaler()

def train_model(dataloader, model, optimizer, intent_loss_fn, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        intent_labels = batch['intent_labels'].to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            intent_logits = model(input_ids, attention_mask)
            intent_loss = intent_loss_fn(intent_logits, intent_labels)

        scaler.scale(intent_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += intent_loss.item()
        progress_bar.set_postfix(loss=intent_loss.item())

    return total_loss / len(dataloader)


from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)

# ============================
# EVALUATION FUNCTION WITH VALIDATION LOSS
# ============================
def evaluate_model(dataloader, model, intent_loss_fn, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            intent_labels = batch['intent_labels'].to(device)

            # Forward pass
            intent_logits = model(input_ids, attention_mask)

            # Compute validation loss
            val_loss = intent_loss_fn(intent_logits, intent_labels)
            total_val_loss += val_loss.item()

            # Get predictions
            preds = torch.argmax(intent_logits, dim=1).cpu().numpy()
            labels = intent_labels.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in range(num_intents)])

    avg_val_loss = total_val_loss / len(dataloader)  # Average validation loss
    return accuracy, f1, avg_val_loss, conf_matrix, report

# ============================
# TRAINING LOOP WITH VALIDATION LOSS
# ============================
for epoch in range(num_epochs):
    # Training phase
    train_loss = train_model(train_dataloader, model, optimizer, intent_loss_fn, device)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")

    # Validation phase
    val_accuracy, val_f1, val_loss, val_conf_matrix, val_report = evaluate_model(val_dataloader, model, intent_loss_fn, device)

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1 Score: {val_f1:.4f}")
    print("Confusion Matrix:")
    print(val_conf_matrix)
    print("Classification Report:")
    print(val_report)


DEvice cuda


C:\Users\reaga\AppData\Local\Temp\ipykernel_5688\1155686904.py:50: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training:   0%|          | 0/1344 [00:00<?, ?it/s]C:\Users\reaga\AppData\Local\Temp\ipykernel_5688\1155686904.py:64: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


KeyboardInterrupt: 

# Distilbert


In [ ]:
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from datasets import Dataset
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
import torch.optim as optim

# Load the dataset from CSV
def load_dataset(csv_file):
    df = pd.read_csv(csv_file)
    if 'instruction_augmented' not in df.columns or 'intent' not in df.columns:
        raise ValueError("Dataset must contain 'instruction_augmented' and 'intent' columns")
    return df

csv_file = "augmented_data4.csv"
print("Loading dataset...")
df = load_dataset(csv_file)
print("Dataset loaded successfully!")

# --- Analysis before dropping ---
print("\n--- Before Dropping ---")
print(f"Total rows: {len(df)}")
print(f"Missing values in 'instruction_augmented': {df['instruction_augmented'].isnull().sum()}")

# --- Drop missing values ---
df.dropna(subset=['instruction_augmented'], inplace=True)

# --- Analysis after dropping ---
print("\n--- After Dropping ---")
print(f"Total rows: {len(df)}")
print(f"Missing values in 'instruction_augmented': {df['instruction_augmented'].isnull().sum()}")

# --- Validation ---
if df['instruction_augmented'].isnull().sum() == 0:
    print("\nValidation: No missing values in 'instruction_augmented' after dropping.")
else:
    print("\nValidation: There are still missing values in 'instruction_augmented'.")

# Display the first few rows to verify
print(df.head())

# Load pre-trained DistilBert tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Load DistilBert model for sequence classification
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['intent'].unique()))

# Move the model to the GPU if available, otherwise, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
distilbert_model.to(device)

# Get the number of unique intents
num_intents = len(df['intent'].unique())
print(f"Number of unique intents: {num_intents}")

# Encode labels for intent classification only
def encode_labels(df):
    # Create a mapping for intent labels
    intent_map = {intent: idx for idx, intent in enumerate(df['intent'].unique())}

    # Reverse mapping for decoding (optional)
    reverse_intent_map = {v: k for k, v in intent_map.items()}

    # Encode 'intent' column
    df['intent_encoded'] = df['intent'].map(intent_map)

    return df, intent_map, reverse_intent_map

# Encode labels
print("Encoding labels...")
df, intent_map, reverse_intent_map = encode_labels(df)

# Print intent mappings
print("\nIntent Mapping:")
print(intent_map)

# Save the preprocessed dataset as JSON
def save_preprocessed_dataset(df, json_file):
    # Convert DataFrame to JSON format
    df.to_json(json_file, orient='records', lines=True)

# Path to save the preprocessed JSON file
json_file = "preprocessed_dataset.json"  # Replace with your desired output path

# Save the preprocessed dataset
print("\nSaving preprocessed dataset as JSON...")
save_preprocessed_dataset(df, json_file)
print(f"Preprocessed dataset saved to {json_file}")

# Load the preprocessed dataset from JSON for intent classification only
print("Loading preprocessed dataset from JSON...")
preprocessed_df = pd.read_json(json_file, lines=True)

# Display the first few rows of the preprocessed dataset
print("\nPreprocessed Dataset:")
print(preprocessed_df.head())

# Verify the encoded 'intent' column
print("\nEncoded Intent Example:")
print(preprocessed_df[['instruction_augmented', 'intent', 'intent_encoded']].head())

# Define a custom dataset class for intent classification
class IntentClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['instruction_augmented']
        intent_label = row['intent_encoded']

        # Tokenize the input
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'intent_labels': intent_label
        }

# Create datasets (train/test split)
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Create dataset objects
train_dataset = IntentClassificationDataset(train_data, tokenizer, max_length=128)
val_dataset = IntentClassificationDataset(val_data, tokenizer, max_length=128)

# Create DataLoader objects
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Compute class weights
intent_weights = compute_class_weight('balanced', classes=np.unique(df['intent_encoded']), y=df['intent_encoded'])
intent_weights = torch.tensor(intent_weights, dtype=torch.float).to(device)

# Define loss function
intent_loss_fn = nn.CrossEntropyLoss(weight=intent_weights)

# Define optimizer
optimizer = optim.AdamW(distilbert_model.parameters(), lr=3e-5)

# Training function
def train_model(dataloader, model, optimizer, intent_loss_fn, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        intent_labels = batch['intent_labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=intent_labels)
        intent_loss = outputs.loss

        intent_loss.backward()
        optimizer.step()

        total_loss += intent_loss.item()
        progress_bar.set_postfix(loss=intent_loss.item())

    return total_loss / len(dataloader)

# Evaluation function
def evaluate_model(dataloader, model, intent_loss_fn, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            intent_labels = batch['intent_labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=intent_labels)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            labels = intent_labels.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in range(num_intents)])

    avg_val_loss = total_val_loss / len(dataloader)
    return accuracy, f1, avg_val_loss, conf_matrix, report

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    # Training phase
    train_loss = train_model(train_dataloader, distilbert_model, optimizer, intent_loss_fn, device)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")

    # Validation phase
    val_accuracy, val_f1, val_loss, val_conf_matrix, val_report = evaluate_model(val_dataloader, distilbert_model, intent_loss_fn, device)

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1 Score: {val_f1:.4f}")
    print("Confusion Matrix:")
    print(val_conf_matrix)
    print("Classification Report:")
    print(val_report)


Loading dataset...
Dataset loaded successfully!

--- Before Dropping ---
Total rows: 26872
Missing values in 'instruction_augmented': 111

--- After Dropping ---
Total rows: 26761
Missing values in 'instruction_augmented': 0

Validation: No missing values in 'instruction_augmented' after dropping.
  category        intent                                           response  \
0    ORDER  cancel_order  I understand the financial constraints you are...   
1    ORDER  cancel_order  I recognize that you've recently made a purcha...   
2    ORDER  cancel_order  I've observed that you no longer want the item...   
3    ORDER  cancel_order  I'm attuned to the idea that you require assis...   
4    ORDER  cancel_order  I realized, you're looking for the information...   

                                instruction_original  \
0          I cannot afford purchase {{Order Number}}   
1  I have bought some item, I have to cancel orde...   
2  I do not want this item, cancel order {{Order ...   
3 

C:\Users\reaga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\reaga\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBer

Number of unique intents: 27
Encoding labels...

Intent Mapping:
{'cancel_order': 0, 'change_order': 1, 'change_shipping_address': 2, 'check_cancellation_fee': 3, 'check_invoice': 4, 'check_payment_methods': 5, 'check_refund_policy': 6, 'complaint': 7, 'contact_customer_service': 8, 'contact_human_agent': 9, 'create_account': 10, 'delete_account': 11, 'delivery_options': 12, 'delivery_period': 13, 'edit_account': 14, 'get_invoice': 15, 'get_refund': 16, 'newsletter_subscription': 17, 'payment_issue': 18, 'place_order': 19, 'recover_password': 20, 'registration_problems': 21, 'review': 22, 'set_up_shipping_address': 23, 'switch_account': 24, 'track_order': 25, 'track_refund': 26}

Saving preprocessed dataset as JSON...
Preprocessed dataset saved to preprocessed_dataset.json
Loading preprocessed dataset from JSON...

Preprocessed Dataset:
  category        intent                                           response  \
0    ORDER  cancel_order  I understand the financial constraints you are

Epoch 1: Train Loss = 0.6397


Validation Loss: 0.2710
Validation Accuracy: 0.9238
Validation F1 Score: 0.9244
Confusion Matrix:
[[180   1   0   0   0   1   0   0   0   0   1   0   0   1   0   0   1   0
    1   1   1   0   0   0   0   4   0]
 [  8 165   0   0   0   0   0   0   0   0   0   0   1   0   0   1   0   0
    0   2   1   0   0   0   1  12   0]
 [  2   2 197   0   0   0   0   2   1   0   1   1   1   1   0   1   0   0
    0   0   1   0   0   9   0   0   0]
 [  7   0   0 185   0   3   0   0   0   0   0   0   3   2   0   0   0   0
    0   1   0   0   1   0   0   1   0]
 [  3   1   0   0 171   0   0   2   1   1   1   1   0   3   0   3   0   1
    0   0   1   0   0   1   1   0   0]
 [  0   0   0   0   0 197   0   0   0   1   1   0   2   0   0   0   0   1
    5   0   2   0   0   0   0   0   0]
 [  0   0   0   0   0   2 171   2   1   1   0   0   2   4   0   0   2   0
    0   1   1   0   0   0   1   0   2]
 [  3   0   0   0   0   2   1 178   1   0   0   0   0   0   0   0   0   0
    1   2   1   0   2   0   0   0   0

...

Validation Loss: 0.2662
Validation Accuracy: 0.9318
Validation F1 Score: 0.9322
Confusion Matrix:
[[174   4   0   0   0   0   0   1   0   0   1   2   3   0   0   0   0   0
    0   2   0   0   0   0   3   1   1]
 [  5 172   1   0   0   0   0   0   0   0   0   0   3   0   2   1   0   0
    0   1   0   0   0   0   1   5   0]
 [  0   0 208   0   0   0   0   0   1   0   2   2   1   0   1   0   0   1
    0   0   1   0   0   2   0   0   0]
 [  0   0   0 190   0   1   1   1   0   0   0   0   2   2   0   0   1   0
    1   0   0   0   2   0   2   0   0]
 [  0   0   2   0 175   0   0   1   1   0   3   0   0   2   0   1   0   0
    0   1   0   0   1   0   3   0   1]
 [  0   0   0   0   0 203   0   0   1   0   0   0   0   0   0   1   0   0
    1   0   1   0   0   0   1   0   1]
 [  0   0   0   0   0   1 174   1   2   0   0   0   2   2   0   0   1   2
    0   1   0   0   1   0   1   0   2]
 [  1   0   1   0   0   0   1 178   5   0   0   0   0   0   0   0   0   0
    0   1   2   0   0   0   2   0   0

Epoch 4: Train Loss = 0.0845


Validation Loss: 0.2905
Validation Accuracy: 0.9326
Validation F1 Score: 0.9334
Confusion Matrix:
[[176   4   0   0   0   0   0   1   0   0   0   4   0   0   0   0   0   2
    0   2   0   0   1   0   0   2   0]
 [  6 175   0   0   0   0   0   0   0   0   0   2   0   0   2   1   0   0
    0   2   0   0   0   0   1   2   0]
 [  0   3 200   0   0   0   0   0   1   1   0   3   0   1   1   0   0   3
    0   0   0   0   0   6   0   0   0]
 [  0   0   0 194   0   0   0   2   0   0   0   0   1   0   0   0   1   2
    0   0   1   0   1   0   0   0   1]
 [  1   1   0   1 169   0   0   2   0   0   0   0   0   0   1   7   0   4
    0   0   0   0   1   1   1   1   1]
 [  1   0   0   0   0 202   0   0   0   1   0   1   0   0   0   0   0   1
    1   0   0   0   0   0   1   0   1]
 [  1   2   0   1   0   0 172   1   0   0   0   0   1   1   0   1   1   3
    0   0   0   0   3   0   0   0   3]
 [  2   0   0   0   0   0   1 179   3   0   0   2   0   0   0   0   0   1
    0   0   0   1   0   0   2   0   0

Epoch 5: Train Loss = 0.0562


Validation Loss: 0.2970
Validation Accuracy: 0.9344
Validation F1 Score: 0.9347
Confusion Matrix:
[[176   8   0   2   0   0   0   1   0   0   0   1   0   0   0   0   1   1
    0   1   0   0   0   0   0   1   0]
 [  4 179   0   1   0   0   0   0   0   0   0   0   0   0   2   1   0   0
    0   1   0   0   0   0   1   1   1]
 [  0   1 207   1   0   0   0   2   1   0   0   0   0   0   2   0   0   1
    0   0   0   0   0   2   1   1   0]
 [  1   0   0 197   0   0   1   2   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   1   0   0   0   0]
 [  0   1   1   3 169   0   1   2   1   1   0   0   0   1   2   6   1   0
    0   1   0   0   0   0   1   0   0]
 [  0   0   1   0   0 203   1   0   0   1   0   0   0   0   1   0   0   0
    1   0   0   0   0   0   0   0   1]
 [  0   0   0   2   0   2 176   1   1   0   0   0   0   1   1   1   0   1
    1   1   0   0   0   0   0   0   2]
 [  1   0   0   1   0   0   1 179   1   0   0   0   0   0   0   0   0   2
    0   1   2   1   0   0   2   0   0